In [1]:
from __future__ import annotations

from typing import  List
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
    
print(f'Current Working Directory: {os.getcwd()}')

Current Working Directory: C:\Users\markm\Documents\programming\Flexible-Cloud-Resource\analysis


In [2]:
from src.extra.visualise import plot_allocation_results, ImageFormat
from src.core.core import reset_model
from src.core.fixed_task import FixedTask, FixedSumPowerSpeeds
from src.core.server import Server
from src.core.task import Task
from src.greedy.greedy import greedy_algorithm
from src.greedy.resource_allocation_policy import SumPowPercentage
from src.greedy.server_selection_policy import ProductResources
from src.greedy.value_density import UtilityDeadlinePerResource
from src.optimal.fixed_optimal import fixed_optimal
from src.optimal.flexible_optimal import flexible_optimal

ImportError: cannot import name 'FixedSumPowerSpeeds' from 'src.core.fixed_task' (C:\Users\markm\Documents\programming\Flexible-Cloud-Resource\src\core\fixed_task.py)

In [ ]:
tasks = [
    Task("Task 1",  required_storage=100, required_computation=100, required_results_data=50, deadline=10, value=100),
    Task("Task 2",  required_storage=75,  required_computation=125, required_results_data=40, deadline=10, value=90),
    Task("Task 3",  required_storage=125, required_computation=110, required_results_data=45, deadline=10, value=110),
    Task("Task 4",  required_storage=100, required_computation=75,  required_results_data=60, deadline=10, value=75),
    Task("Task 5",  required_storage=85,  required_computation=90,  required_results_data=55, deadline=10, value=125),
    Task("Task 6",  required_storage=75,  required_computation=120, required_results_data=40, deadline=10, value=100),
    Task("Task 7",  required_storage=125, required_computation=100, required_results_data=50, deadline=10, value=80),
    Task("Task 8",  required_storage=115, required_computation=75,  required_results_data=55, deadline=10, value=110),
    Task("Task 9",  required_storage=100, required_computation=110, required_results_data=60, deadline=10, value=120),
    Task("Task 10", required_storage=90,  required_computation=120, required_results_data=40, deadline=10, value=90),
    Task("Task 11", required_storage=110, required_computation=90,  required_results_data=45, deadline=10, value=100),
    Task("Task 12", required_storage=100, required_computation=80,  required_results_data=55, deadline=10, value=100)
]

servers = [
    Server("Server 1", storage_capacity=500, computation_capacity=100, bandwidth_capacity=220),
    Server("Server 2", storage_capacity=500, computation_capacity=100, bandwidth_capacity=210),
    Server("Server 3", storage_capacity=500, computation_capacity=90, bandwidth_capacity=250)
]

In [ ]:
def print_tasks(tasks: List[Task]):
    max_task_name_len = max(len(task.name) for task in tasks) + 1
    print("{:<{}}| Value |{:^9}|{:^13}|{:^9}|{:^10}|{:^9}|{:^9}|{:^9}| {}"
          .format("Name", max_task_name_len, "Storage", "Computation", "Results", "Deadline", 
                  "Loading", "Compute", "Sending", "Server"))
    
    for task in tasks:
        print("{:<{name_len}}|{:^7.1f}|{:^9}|{:^13}|{:^9}|{:^10}|{:^9}|{:^9}|{:^9}|{:^10}"
              .format(task.name, task.value, task.required_storage, task.required_computation, task.required_results_data,
                      task.deadline, task.loading_speed, task.compute_speed, task.sending_speed,
                      task.running_server.name if task.running_server else "None", name_len=max_task_name_len))

### Optimal Solution

In [ ]:
optimal_results = flexible_optimal(tasks, servers, time_limit=5)
minimise_resource_allocation(tasks, servers)

print_tasks(tasks)
plot_allocation_results(tasks, servers, "Optimal Flexible Resource Allocation")
reset_model(tasks, servers)

### Fixed Solution

In [ ]:
fixed_tasks = [FixedTask(task, FixedSumPowerSpeeds(), fixed_name=False) for task in tasks]
fixed_results = fixed_optimal(fixed_tasks, servers, time_limit=5)

print_tasks(fixed_tasks)
plot_allocation_results(fixed_tasks, servers, "Optimal Fixed Resource Allocation")
reset_model(tasks, servers)

### Greedy Solution

In [ ]:
greedy_results = greedy_algorithm(tasks, servers, UtilityDeadlinePerResource(), ProductResources(), SumPowPercentage())

print_tasks(tasks)
plot_allocation_results(tasks, servers, "Greedy Flexible Resource Allocation")

### Optimal, Fixed and Greedy results

In [ ]:
results = {'Optimal': optimal_results, 'Fixed': fixed_results, 'Greedy': greedy_results}

max_name_len = max(len(name) for name in results.keys())
max_storage_len = max(len(str(list(result.server_storage_used.values()))) for result in results.values())
max_computation_len = max(len(str(list(result.server_computation_used.values()))) for result in results.values())
max_bandwidth_len = max(len(str(list(result.server_bandwidth_used.values()))) for result in results.values())

print(f"{'Name':<{max_name_len}} | Social Welfare | {'Storage':^{max_storage_len}} | {'Computation':^{max_computation_len}} | "
      f"{'Bandwidth':^{max_bandwidth_len}} | Num Jobs")
for name, result in results.items():
    print(f"{name:<{max_name_len}} | {result.social_welfare:^5} | "
          f"{str(list(result.server_storage_used.values())):^{max_storage_len}} | "
          f"{str(list(result.server_computation_used.values())):^{max_computation_len}} | "
          f"{str(list(result.server_bandwidth_used.values())):^{max_bandwidth_len}} | "
          f"{str(list(result.server_num_tasks_allocated.values()))}")